# Setting up Observability for Public Cloud managed Infrastructure

When you provision connection objects, you enable authentication and authorization for AppDynamics Cloud with your cloud provider. AppDynamics Cloud supports Amazon Web Services (AWS) Role Delegation and AWS Access Key Credentials. In this section, we use connection endpoints that initiate the AppDynamics data collection.

Let's create cloud connection leveraging API's here. Refer to the [API Reference](https://developer.cisco.com/docs/appdynamics/cloud-connection/#!api-reference).

Before creating connections, we will create configuration objects. If you want to explore the lifecycle of configuration and connection objects, refer to this learning lab:
(https://developer.cisco.com/learning/labs/cloudconnections-jupyter/introduction/)
    
To progress with the lab, click **inside** the boxes below that contain the snippets of Python code. Note that the **Run** button will appear to the left of the code box. Click on the **arrow** button to execute the code snippet. After you click **Run**, executions in progress will be denoted by "ln[*]"

> **Note:** Do not skip code blocks. Execute each before you progress to the next.

> **Note:** Wait for the entire execution to complete in a code block before you progress to the next.

## Pre-execution steps

Recall the API token created in the previous notebook. Click **Run** to set the environment.

In [ ]:
import os
%store -r appd_token
%store -r base_url
%store -r aws_key
%store -r aws_secret
%store -r aws_conn_name
%store -r aws_conf_name

os.environ['APPD_TOK'] = appd_token
os.environ['BASE_URL'] = base_url
os.environ['AWS_KEY'] = aws_key
os.environ['AWS_SECRET'] = aws_secret
os.environ['AWS_CONN_NAME'] = aws_conn_name
os.environ['AWS_CONF_NAME'] = aws_conf_name
print("Globals set:" + "\n")
print("API Token:" + appd_token + "\n")
print("Base URL:" + base_url + "\n")
print("Configuration Name:" + aws_conf_name + "\n")
print("Connection Name:" + aws_conn_name + "\n")
print("AWS Secret:" + aws_secret + "\n")
print("AWS Key:" + aws_key + "\n")


## Create Connection objects

AppDynamics Cloud supports Amazon Web Services (AWS) Role Delegation, AWS Access Key Credentials to authenticate and authorize AppDynamics Cloud with AWS. Here, we use AWS Access Key methodology to create the cloud connection to AWS.

### Creating AWS connections with AWS Access Key

In [ ]:
import os, requests, json

def create_aws_config(appd_token, base_url, conf_name):
    url = base_url + "/cloud/v1/configurations"
    headers = {
        'Content-Type': 'application/json',
        'Accept': '*/*',
        'Authorization': 'Bearer ' + appd_token
    }
    data = {
        "displayName": conf_name,
        "type": "aws",
        "details": {
            "regions": [],
            "polling": {
                "interval": 5,
                "unit": "minute"
            },
            "services": [{
                "name": "ec2",
                "polling": {
                    "interval": 5,
                    "unit": "minute"
                }
            }],
            "importTags": {
                "enabled": "true",
                "excludedKeys": []
            }
        }
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(data))
    if response.ok:
        json_object = json.loads(response.text)
        print("Retrieved Configuration:")
        print(json.dumps(json_object, indent = 3)) 
        token_json = response.json()
        cid = token_json['id']
        return(cid)
    else:
        print ("Could not create Configuration object. May already exist.")
        return None


def create_aws_connection_key(appd_token, base_url, configid, accesskey, secretkey, name):
    url = base_url + "/cloud/v1/connections"
    headers = {
        'Content-Type': 'application/json',
        'Accept': '*/*',
        'Authorization': 'Bearer ' + appd_token
    }
    data = {
        "displayName": name,
        "description": "Description for this AWS access key connection without assigned configurationId",
        "type": "aws",
        "details": {
            "accessKeyId": "" + accesskey,
            "secretAccessKey": "" + secretkey,
            "accessType": "access_key"
        },
        "configurationId": configid
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(data))
    if response.ok:
        print("\n Created Connection:")
        json_object = json.loads(response.text)
        print(json.dumps(json_object, indent = 3)) 
    else:
        json_object = json.loads(response.text)
        print ("Could not create Connection object. Connection may already exist for this AWS account or reservation may have ended. Please check your reservation is still active.")
    
appd_token = os.getenv('APPD_TOK')
base_url = os.getenv('BASE_URL')
aws_key = os.getenv('AWS_KEY')
aws_secret = os.getenv('AWS_SECRET')
conn_name = os.getenv('AWS_CONN_NAME')
conf_name = os.getenv('AWS_CONF_NAME')
configid = create_aws_config(appd_token, base_url, conf_name)
if not configid:
    print ("Cannot create connection.")
else:
    create_aws_connection_key(appd_token, base_url, configid, aws_key, aws_secret, conn_name)